In [ ]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image, ImageEnhance ,ImageFilter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
df_pov = pd.read_csv('../datos/datos_bo3_pov.csv')

In [ ]:
df_pov.head()

In [ ]:
print(df_pov.shape)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i in range(0,5):
    primera_fila = df_pov.iloc[0]
    imagen = Image.open("../" +str(primera_fila.iloc[i]))
    matriz_np = np.array(imagen)  

    axes[i].imshow(matriz_np)
    axes[i].axis('off')

plt.tight_layout()
plt.show()  

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(15, 5))

for i in range(3): 
    for j in range(5):
        # Obtener la ruta de la imagen
        ruta_imagen = df_pov.iloc[i, j]  # Elimina los corchetes
        
        # Abrir la imagen y convertirla a matriz numpy
        imagen = Image.open("../" + ruta_imagen)
        matriz_np = np.array(imagen)
        
        # Mostrar la imagen en el subplot correspondiente
        axes[i, j].imshow(matriz_np)
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
primera_fila = df_pov.iloc[0]
imagen = Image.open("../" +str(primera_fila.iloc[1]))
matriz_np = np.array(imagen)  
print("Tamaño imagen -> "+str(imagen.size))
print("Tamaño matriz -> "+ str(matriz_np.shape))
plt.imshow(matriz_np)
plt.axis('on') 
plt.show()

In [ ]:
def comprobar_contenido(X_train , Y_train , num_video):

    fig, axs = plt.subplots(1, 5, figsize=(5 * 2, 1 * 2))

    for i in range(5):
        axs[i].imshow(X_train[num_video][i])
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()
    print("Etiqueta -> "+ str(Y_train[num_video]))
    return Y_train[num_video]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.layers import Input, TimeDistributed, MaxPooling2D, Flatten, Dense ,ConvLSTM2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import Sequence

In [ ]:
class ImageDataGenerator(Sequence):
    def __init__(self, df , sequence_length ,image_height , image_width , batch_size=32, **kwargs):
        self.df = df
        self.image_sequence_length = sequence_length
        self.image_height = image_height
        self.image_width = image_width
        self.batch_size = batch_size
        super().__init__(**kwargs)  # Llamada al constructor de la clase base
        
    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))
    
    def __getitem__(self, index):
        inicio_lote = index * self.batch_size
        final_lote = (index + 1) * self.batch_size
        
        batch_df = self.df.iloc[inicio_lote:final_lote]     #Obtenemos un lote del df
        
        X_batch = np.zeros((len(batch_df), self.image_sequence_length ,self.image_height, self.image_width, 3))  # Inicializar matriz para las imágenes
        y_batch = np.array([eval(mouse_final) for mouse_final in batch_df['mouse_final']])          #Almacenamos todas las label del df que corresponden al batch
        
        for i, (_, row) in enumerate(batch_df.iterrows()):
            frames = []
            for j in range(self.image_sequence_length):
                imagen = Image.open("../" + row.iloc[j])    # Obtener la ruta de la imagen
                imagen = Image.fromarray(np.array(imagen)).filter(ImageFilter.SHARPEN)  #Aplicamos Filtro SHARPEN
                imagen = np.array(imagen) / 255.0           # Normalizar la imagen

                frames.append(imagen)                  #Juntamos frames en lista
            video = np.stack(frames, axis=0)           # Convertir lista de frames en un video np
            X_batch[i] = video
        
        return X_batch, y_batch

# Preparacion Entrenamiento

In [ ]:
# Modelo Pov
df_train_pov, df_test_pov = train_test_split(df_pov, test_size=0.2, random_state=42)
df_train_pov, df_valid_pov = train_test_split(df_train_pov, test_size=0.25, random_state=42)

print("Tamaño total: ",len(df_pov))
print("Tamaño del conjunto de entrenamiento:", len(df_train_pov))
print("Tamaño del conjunto de validación:", len(df_valid_pov))
print("Tamaño del conjunto de prueba:", len(df_test_pov))

In [ ]:
altura_img = 93
anchura_img = 135
batch_size = 4

In [ ]:
# Generadores de Batches POV
train_generator_pov = ImageDataGenerator(df_train_pov, sequence_length=5, image_height=altura_img, image_width=anchura_img, batch_size=batch_size)
valid_generator_pov = ImageDataGenerator(df_valid_pov, sequence_length=5, image_height=altura_img, image_width=anchura_img, batch_size=batch_size)